## Vietnam Public Company Financial Information Crawler
##### Last Update: 2022-08-02
#### Author: Wonho Lim 
#### E-mail: wonholim02@gmail.com
#### Python Version: Python 3.10.4 (ipykernel)
#### Chrome Version: Chrome 103.0.5060.134 (64-bit)
#### Chrome Driver Version: ChromeDriver 103.0.5060.134

#### Crawled Website
Investing.com MAIN: https://www.investing.com /
Investing.com Vietnam: https://www.investing.com/equities/vietnam

Description: This is public company financial statement web crawling code for investing.com, especially for companies in Vietnam. investing.com is an unofficial stock information website for countries around the world. Even though it is an unofficial website, comparably, it has decent amount of information including key financial values. However, the server itself is very unstable that loading error occurs a lot, and data table format/language format is not standardized that a lot of exceptions are made(goes to failList). So one should be aware of that, and it is better to find alternative source for a long term development. Using excel format instead of csv could be a solution, which I have not tried so far. This code can be reused for other countries by editing initial website address and several page setting button XPATH locations - Colombia, Indonesia, Italy, Sinbgapore, and Spain is already written and uploaded. For crawling, as long as the website has not been modified, the code below must be run properly if and only if it is run in an order. Moreover, environment setting must be fixed based on user's computer/server setting and location.

### 1. Importing useful open source librabries - utilized BS4 and Selenium Web Driver for crawling

In [1]:
import bs4
import time
import csv 
import pandas as pd 
from platform import python_version
import requests
import lxml 
import xlrd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By

### 2. Environment
#### - Chromdriver must be located in local/cloud PC folder.
#### - One can download proper version at: chromedriver.chromium.org/downloads
#### - Path should be modified based on local environment

### 3. Main Crawler
#### - Uncollectable or unavailable data was collected as "" for convenience.
#### - Data Cleansing might be required after crawling sometiems, but it will not be tough as one knows how to use excel.
#### - Some directions and explanations are written as comment below.
#### - The order of append must be depend on the order of column name assigned above.
#### - Lots of try-except function is used to avoid error caused by non-existing pages or information.
#### - One also need to avoid ad and accept cookie banner that randomly appears.
#### - Run the crawler many times to improve the accuracy - Detail is written below main crawler.

### 2020

In [ ]:
cover = open('Vietnam2020.csv','w', newline='', encoding='CP949')
path = r'C:\Users\user\Desktop\자료\chromedriver'
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])

# Basic Setting
info = []

# Get EURONEXT Brussels Stock Website
driver = webdriver.Chrome(path)
base_url = "https://www.investing.com"
driver.get('https://www.investing.com/equities/vietnam')
# Avoid Ad
try:
    driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
except:
    print("pass")
time.sleep(1)
driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[6]/select').click()
time.sleep(2)
driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[6]/select/option[1]').click()
time.sleep(10)

# Get each company's link using href in a tag
text = driver.page_source
time.sleep(2)
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='cross_rate_markets_stocks_1')
time.sleep(2)
all_atag_maintable = maintable.find_all('a')
print(maintable)
company_links = []

# We have to process the collected data again because of their data structure
for a in all_atag_maintable:
    company_link = a.attrs["href"]
    company_links.append(company_link)
failList = []
# Main Data Collection code
for sub in company_links:
    try:
        # crawling environment setting
        info = []
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        time.sleep(1)
        try:
            driver.get(base_url + sub)
            time.sleep(4)
        except:
            try:
                driver.get("https://www.google.com/")
                time.sleep(2)
                driver.get(base_url + sub)
                time.sleep(3)
            except:
                driver.get(base_url + sub)
                time.sleep(6)
        info = []
        
        # Company Name
        name = driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text
        # Counry Information - Must be investigated individually
        info.append("VNM") 
        info.append("Vietnam") 
        info.append("Vietnam") 
        info.append("UTC+07:00") 
        info.append("아시아") 
        info.append("1047000000000 USD") 
        info.append("96208984") 
        info.append("동남아시아") 

        # Unique Code
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text) 
        except:
            info.append("VNMHBR" + name) 
        info.append(name)
        info.append(name)
        
        #Descripton, Contact, Address, Extra Information, Management Information
        time.sleep(1)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]/a').click()
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        try:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Vietnam Stock Exchange."
            description_long = info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/p').text) 
        except:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Vietnam Stock Exchange."
            description_long =  name + "(English: " + name + ")" + " is a public company that is listed on Vietnam Stock Exchange. They are providing products/services in Vietnam, and they are operated as public company."

        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Vietnam"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[2]').text + ", Vietnam, Asia"
            post = address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[3]').text
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Vietnam"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Vietnam, Asia"
                post = "N/A"
            except:
                address1 = "Vietnam"
                address2 = "Vietnam, Asia"
                post = "N/A"
        info.append(description_short)
        info.append(description_short)
        info.append(description_long)
        info.append(description_long)
        info.append("")
        info.append("")
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[4]/span[3]/a').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(address1)
        info.append(address1)
        info.append(address2)
        info.append(address2)
        info.append(post) 

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[3]/p').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("Listed") 
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[1]').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        info.append("Executive Board")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
            

        # Financial Information
        try:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[3]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[4]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        time.sleep(3)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(3)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #회계연도
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[3]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[3]/span').text) 
            except:
                info.append("")
        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text + "M. VND ")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text  + "M. VND ")
            except:
                info.append("")
        #비유동자산금액
        info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text + "M. VND ")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text  + "M. VND")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[3]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[3]').text  + "M. VND")
            except:
                info.append("")
        #비유동부채금액
        info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[3]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[3]').text + "M. VND")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[3]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[3]').text + "M. VND")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text  + "M. VND")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[2]/a').click()
        time.sleep(2)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text + "M. VND")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text + "M. VND")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[3]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[3]').text + "M. VND")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[3]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. VND")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[3]').text + "M. VND")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[3]').text + "M. VND")
            except:
                info.append("")
        #금융비용금액
        info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. VND")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[3]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[3]').text + "M. VND")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[3]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[3]').text + "M. VND")
            except:
                info.append("")

        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[4]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(5)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[3]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[3]').text + "M. VND")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[3]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[3]').text + "M. VND")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[3]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[3]').text + "M. VND")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[3]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[3]').text + "M. VND")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. VND")
            except:
                info.append("")
        #재무정보중 계산예정
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        # Industry and Products
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[1]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]').click()
        time.sleep(5)

        try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            except:
                info.append("Public Company")
                info.append("Public Company")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
            except:
                info.append("Product and Services")
                info.append("Product and Services")


        # Language Information
        info.append("VI")
        info.append("Vietnamese")
        info.append("Tieng Viet")

        # Stock Market Information
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[1]/a').click()
        time.sleep(2)
        info.append("VSE")
        info.append("Vietnam Stock Exchange")
        info.append("Vietnam Stock Exchange")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text)
            except:
                info.append("")
        #상장일자
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[2]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[5]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        #거래량
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[7]/dd/span/span[1]').text)
        except:
            info.append("")
        #시가총액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[8]/dd').text + "VND")
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Vietnam")
        info.append("Vietnam, Asia")
        info.append("")
        info.append("")

        # Currency Information
        info.append("")

        #화폐
        info.append("VND")
        info.append("Vietnamese Dong")

        # Management
        info.append("Chris")
        info.append("Investing.com")
        info.append("2022-07-04")

        # For Status Checking
        print(info)

        # Storing Data on CSV
        writing.writerow(info)
    
    #if fails
    except:
        print("fail")
        failList.append(base_url + sub)
            
# Check failed result 
# - There are many ways to deal with failList: retry, ignore, manually add, find why error has occured.
# - This will be up to user's choice.
print("failLength:")
print(len(failList))
print(failList)

C:\Users\user\AppData\Local\Temp\ipykernel_23636\1233140420.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


pass
<table class="genTbl closedTbl crossRatesTbl elpTbl elp25" id="cross_rate_markets_stocks_1" tablesorter=""><thead><tr><th class="flag"> </th><th class="left noWrap elp pointer">Name<span class="headerSortDefault newSiteIconsSprite" sort_default=""></span></th><th>Last</th><th>High</th><th>Low</th><th data-col-caption="Change">Chg.</th><th class="pointer" data-col-caption="Change %">Chg. %<span class="headerSortDefault newSiteIconsSprite" sort_default="" sortablecache=""></span></th><th class="pointer" data-col-caption="Vol">Vol.<span class="headerSortUp newSiteIconsSprite" sort_default="" sortablecache=""></span></th><th class="pointer" data-col-caption="Time">Time<span class="headerSortDefault newSiteIconsSprite" sort_default="" sortablecache=""></span></th><th class="icon"></th></tr></thead><tbody><tr id="pair_958514"><td class="flag"><span class="ceFlags Vietnam" title="Vietnam"> </span></td><td class="bold left noWrap elp plusIconTd"><a href="/equities/hoa-phat-group-jsc" titl

fail
pass
fail
pass
pass
pass
pass
fail
pass
fail
pass
fail
pass
fail
pass
pass
pass
pass
Info Length:
111
pass
pass
pass
fail
pass
pass
pass
pass
Info Length:
111
pass
pass
pass
pass
Info Length:
111
pass
pass
pass
pass
Info Length:
111
pass
fail
pass
pass
pass


### 2019

In [ ]:
cover = open('Vietnam2019.csv','w', newline='', encoding='CP949')
path = r'C:\Users\user\Desktop\자료\chromedriver'
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])

# Basic Setting
info = []

# Get EURONEXT Brussels Stock Website
driver = webdriver.Chrome(path)
base_url = "https://www.investing.com"
driver.get('https://www.investing.com/equities/vietnam')
# Avoid Ad
try:
    driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
except:
    print("pass")
time.sleep(1)
driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[6]/select').click()
time.sleep(2)
driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[6]/select/option[1]').click()
time.sleep(10)

# Get each company's link using href in a tag
text = driver.page_source
time.sleep(2)
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='cross_rate_markets_stocks_1')
time.sleep(2)
all_atag_maintable = maintable.find_all('a')
print(maintable)
company_links = []
failList = []

# We have to process the collected data again because of their data structure
for a in all_atag_maintable:
    company_link = a.attrs["href"]
    company_links.append(company_link)

# Main Data Collection code
for sub in company_links:
    try:
        # crawling environment setting
        info = []
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        time.sleep(1)
        try:
            driver.get(base_url + sub)
            time.sleep(4)
        except:
            try:
                driver.get("https://www.google.com/")
                time.sleep(2)
                driver.get(base_url + sub)
                time.sleep(3)
            except:
                driver.get(base_url + sub)
                time.sleep(6)
        info = []
        
        # Company Name
        name = driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text
        # Counry Information - Must be investigated individually
        info.append("VNM") 
        info.append("Vietnam") 
        info.append("Vietnam") 
        info.append("UTC+07:00") 
        info.append("아시아") 
        info.append("1047000000000 USD") 
        info.append("96208984") 
        info.append("동남아시아") 

        # Unique Code
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text) 
        except:
            info.append("VNMHBR" + name) 
        info.append(name)
        info.append(name)
        
        #Descripton, Contact, Address, Extra Information, Management Information
        time.sleep(1)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]/a').click()
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        try:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Vietnam Stock Exchange."
            description_long = info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/p').text) 
        except:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Vietnam Stock Exchange."
            description_long =  name + "(English: " + name + ")" + " is a public company that is listed on Vietnam Stock Exchange. They are providing products/services in Vietnam, and they are operated as public company."

        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Vietnam"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[2]').text + ", Vietnam, Asia"
            post = address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[3]').text
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Vietnam"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Vietnam, Asia"
                post = "N/A"
            except:
                address1 = "Vietnam"
                address2 = "Vietnam, Asia"
                post = "N/A"
        info.append(description_short)
        info.append(description_short)
        info.append(description_long)
        info.append(description_long)
        info.append("")
        info.append("")
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[4]/span[3]/a').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(address1)
        info.append(address1)
        info.append(address2)
        info.append(address2)
        info.append(post) 

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[3]/p').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("Listed") 
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[1]').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        info.append("Executive Board")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
            

        # Financial Information
        try:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[3]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[4]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        time.sleep(3)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(3)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #회계연도
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[4]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[3]/span').text) 
            except:
                info.append("")
        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[4]').text + "M. VND ")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text  + "M. VND ")
            except:
                info.append("")
        #비유동자산금액
        info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[4]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text  + "M. VND")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[4]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[3]').text  + "M. VND")
            except:
                info.append("")
        #비유동부채금액
        info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[4]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[3]').text + "M. VND")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[4]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[3]').text + "M. VND")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[4]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text  + "M. VND")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[2]/a').click()
        time.sleep(2)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[4]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text + "M. VND")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[4]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text + "M. VND")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[4]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[3]').text + "M. VND")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[4]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. VND")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[4]').text + "M. VND")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[3]').text + "M. VND")
            except:
                info.append("")
        #금융비용금액
        info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[4]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. VND")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[4]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[3]').text + "M. VND")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[4]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[3]').text + "M. VND")
            except:
                info.append("")

        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[4]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(5)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[4]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[3]').text + "M. VND")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[4]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[3]').text + "M. VND")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[4]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[3]').text + "M. VND")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[4]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[3]').text + "M. VND")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[4]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. VND")
            except:
                info.append("")
        #재무정보중 계산예정
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        # Industry and Products
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[1]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]').click()
        time.sleep(5)

        try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            except:
                info.append("Public Company")
                info.append("Public Company")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
            except:
                info.append("Product and Services")
                info.append("Product and Services")


        # Language Information
        info.append("VI")
        info.append("Vietnamese")
        info.append("Tieng Viet")

        # Stock Market Information
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[1]/a').click()
        time.sleep(2)
        info.append("VSE")
        info.append("Vietnam Stock Exchange")
        info.append("Vietnam Stock Exchange")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text)
            except:
                info.append("")
        #상장일자
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[2]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[5]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        #거래량
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[7]/dd/span/span[1]').text)
        except:
            info.append("")
        #시가총액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[8]/dd').text + "VND")
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Vietnam")
        info.append("Vietnam, Asia")
        info.append("")
        info.append("")

        # Currency Information
        info.append("")

        #화폐
        info.append("VND")
        info.append("Vietnamese Dong")

        # Management
        info.append("Chris")
        info.append("Investing.com")
        info.append("2022-07-04")

        # For Status Checking
        print(info)

        # Storing Data on CSV
        writing.writerow(info)
    
    #if fails
    except:
        print("fail")
        failList.append(base_url + sub)
            
# Check failed result 
# - There are many ways to deal with failList: retry, ignore, manually add, find why error has occured.
# - This will be up to user's choice.
print("failLength:")
print(len(failList))
print(failList)

### 2018

In [ ]:
cover = open('Vietnam2018.csv','w', newline='', encoding='CP949')
path = r'C:\Users\user\Desktop\자료\chromedriver'
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])

# Basic Setting
info = []

# Get EURONEXT Brussels Stock Website
driver = webdriver.Chrome(path)
base_url = "https://www.investing.com"
driver.get('https://www.investing.com/equities/vietnam')
# Avoid Ad
try:
    driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
except:
    print("pass")
time.sleep(1)
driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[6]/select').click()
time.sleep(2)
driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[6]/select/option[1]').click()
time.sleep(10)
failList = []

# Get each company's link using href in a tag
text = driver.page_source
time.sleep(2)
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='cross_rate_markets_stocks_1')
time.sleep(2)
all_atag_maintable = maintable.find_all('a')
print(maintable)
company_links = []

# We have to process the collected data again because of their data structure
for a in all_atag_maintable:
    company_link = a.attrs["href"]
    company_links.append(company_link)

# Main Data Collection code
for sub in company_links:
    try:
        # crawling environment setting
        info = []
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        time.sleep(1)
        try:
            driver.get(base_url + sub)
            time.sleep(4)
        except:
            try:
                driver.get("https://www.google.com/")
                time.sleep(2)
                driver.get(base_url + sub)
                time.sleep(3)
            except:
                driver.get(base_url + sub)
                time.sleep(6)
        info = []
        
        # Company Name
        name = driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text
        # Counry Information - Must be investigated individually
        info.append("VNM") 
        info.append("Vietnam") 
        info.append("Vietnam") 
        info.append("UTC+07:00") 
        info.append("아시아") 
        info.append("1047000000000 USD") 
        info.append("96208984") 
        info.append("동남아시아") 

        # Unique Code
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text) 
        except:
            info.append("VNMHBR" + name) 
        info.append(name)
        info.append(name)
        
        #Descripton, Contact, Address, Extra Information, Management Information
        time.sleep(1)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]/a').click()
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        try:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Vietnam Stock Exchange."
            description_long = info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/p').text) 
        except:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Vietnam Stock Exchange."
            description_long =  name + "(English: " + name + ")" + " is a public company that is listed on Vietnam Stock Exchange. They are providing products/services in Vietnam, and they are operated as public company."

        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Vietnam"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[2]').text + ", Vietnam, Asia"
            post = address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[3]').text
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Vietnam"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Vietnam, Asia"
                post = "N/A"
            except:
                address1 = "Vietnam"
                address2 = "Vietnam, Asia"
                post = "N/A"
        info.append(description_short)
        info.append(description_short)
        info.append(description_long)
        info.append(description_long)
        info.append("")
        info.append("")
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[4]/span[3]/a').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(address1)
        info.append(address1)
        info.append(address2)
        info.append(address2)
        info.append(post) 

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[3]/p').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("Listed") 
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[1]').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        info.append("Executive Board")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
            

        # Financial Information
        try:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[3]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[4]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        time.sleep(3)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(3)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #회계연도
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[5]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[3]/span').text) 
            except:
                info.append("")
        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[5]').text + "M. VND ")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text  + "M. VND ")
            except:
                info.append("")
        #비유동자산금액
        info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[5]').text + "M. VND ")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text  + "M. VND")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[5]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[3]').text  + "M. VND")
            except:
                info.append("")
        #비유동부채금액
        info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[5]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[3]').text + "M. VND")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[5]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[3]').text + "M. VND")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[5]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text  + "M. VND")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[2]/a').click()
        time.sleep(2)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[5]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text + "M. VND")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[5]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text + "M. VND")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[5]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[3]').text + "M. VND")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[5]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. VND")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[5]').text + "M. VND")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[3]').text + "M. VND")
            except:
                info.append("")
        #금융비용금액
        info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[5]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. VND")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[5]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[3]').text + "M. VND")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[5]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[3]').text + "M. VND")
            except:
                info.append("")

        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[4]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(5)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[5]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[3]').text + "M. VND")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[5]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[3]').text + "M. VND")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[5]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[3]').text + "M. VND")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[5]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[3]').text + "M. VND")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[5]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. VND")
            except:
                info.append("")
        #재무정보중 계산예정
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        # Industry and Products
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[1]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]').click()
        time.sleep(5)

        try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            except:
                info.append("Public Company")
                info.append("Public Company")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
            except:
                info.append("Product and Services")
                info.append("Product and Services")


        # Language Information
        info.append("VI")
        info.append("Vietnamese")
        info.append("Tieng Viet")

        # Stock Market Information
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[1]/a').click()
        time.sleep(2)
        info.append("VSE")
        info.append("Vietnam Stock Exchange")
        info.append("Vietnam Stock Exchange")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text)
            except:
                info.append("")
        #상장일자
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[2]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[5]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        #거래량
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[7]/dd/span/span[1]').text)
        except:
            info.append("")
        #시가총액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[8]/dd').text + "VND")
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Vietnam")
        info.append("Vietnam, Asia")
        info.append("")
        info.append("")

        # Currency Information
        info.append("")

        #화폐
        info.append("VND")
        info.append("Vietnamese Dong")

        # Management
        info.append("Chris")
        info.append("Investing.com")
        info.append("2022-07-04")

        # For Status Checking
        print(info)

        # Storing Data on CSV
        writing.writerow(info)
    
    #if fails
    except:
        print("fail")
        failList.append(base_url + sub)
            
# Check failed result 
# - There are many ways to deal with failList: retry, ignore, manually add, find why error has occured.
# - This will be up to user's choice.
print("failLength:")
print(len(failList))
print(failList)

### 4. Dealing with FailList
#### - Because Investing.com causes loading time error due to unstable server status, it is better to run the code, at least with companies in failList, for several times or run with two or more computers. That will increase the sucess rate much higher than running the code once. When running a new code, make sure to change the name of saving file so that you don't replace the original file.
#### - There are many ways to deal with elements failList: retry(if caused by server loading status), ignore(if caused by actually non-existing data), manually add(if only few exist), find why error has occurred(if massive).
#### - Below is both actual and sample code that was used in 2022

In [ ]:
print(len(failList))
print(failList)

In [10]:
cover = open('Vietnam_2022_investing_2022-07-04_Chris - 2.csv','w', newline='', encoding='CP949')
path = r'C:\Users\user\Desktop\자료\chromedriver'
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])

791

In [11]:
failList2 = []
info = []
driver = webdriver.Chrome(path)
base_url = "https://www.investing.com"
driver.get('https://www.investing.com/equities/vietnam')
time.sleep(2)
try:
    driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
except:
    print("pass")
time.sleep(1)
    
for sub in failList:
    try:
        info = []
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        time.sleep(1)
        driver.get(sub)
        info = []
        time.sleep(4)
        #이름
        name = driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text
        #국가코드
        info.append("VNM") 
        info.append("Vietnam") 
        info.append("Vietnam") 
        info.append("UTC+07:00") 
        info.append("아시아") 
        info.append("1047000000000 USD") 
        info.append("96208984") 
        info.append("동남아시아") 

        #기업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text) 
        except:
            info.append("VNMHBR" + name) 
        info.append(name)
        info.append(name)
        time.sleep(1)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]/a').click()
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        try:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Vietnam Stock Exchange."
            description_long = info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/p').text) 
        except:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Vietnam Stock Exchange."
            description_long =  name + "(English: " + name + ")" + " is a public company that is listed on Vietnam Stock Exchange. They are providing products/services in Vietnam, and they are operated as public company."

        #기업 + 기업연락처 + 기업주소 + 기업추가정보 + 담당자정보
        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Vietnam"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[2]').text + ", Vietnam, Asia"
            post = address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[3]').text
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Vietnam"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Vietnam, Asia"
                post = "N/A"
            except:
                address1 = "Vietnam"
                address2 = "Vietnam, Asia"
                post = "N/A"
        info.append(description_short)
        info.append(description_short)
        info.append(description_long)
        info.append(description_long)
        info.append("")
        info.append("")
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[4]/span[3]/a').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(address1)
        info.append(address1)
        info.append(address2)
        info.append(address2)
        info.append(post) 

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[3]/p').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("Listed") 
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[1]').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        info.append("Executive Board")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #재무정보
        try:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[3]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[4]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        time.sleep(3)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(3)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #회계연도
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[2]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[3]/span').text) 
            except:
                info.append("")
        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. VND ")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text  + "M. VND ")
            except:
                info.append("")
        #비유동자산금액
        info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. VND ")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text  + "M. VND")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[3]').text  + "M. VND")
            except:
                info.append("")
        #비유동부채금액
        info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[3]').text + "M. VND")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[3]').text + "M. VND")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text  + "M. VND")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[2]/a').click()
        time.sleep(2)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text + "M. VND")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text + "M. VND")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[3]').text + "M. VND")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. VND")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[2]').text + "M. VND")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[3]').text + "M. VND")
            except:
                info.append("")
        #금융비용금액
        info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. VND")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[3]').text + "M. VND")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[3]').text + "M. VND")
            except:
                info.append("")

        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[4]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(5)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[3]').text + "M. VND")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[3]').text + "M. VND")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[3]').text + "M. VND")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[3]').text + "M. VND")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. VND")
            except:
                info.append("")
        #재무정보중 계산예정
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        #산업군
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[1]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]').click()
        time.sleep(5)

        try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            except:
                info.append("Public Company")
                info.append("Public Company")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
            except:
                info.append("Product and Services")
                info.append("Product and Services")


        #언어
        info.append("VI")
        info.append("Vietnamese")
        info.append("Tieng Viet")

        #주식시장정보
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[1]/a').click()
        time.sleep(2)
        info.append("VSE")
        info.append("Vietnam Stock Exchange")
        info.append("Vietnam Stock Exchange")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text)
            except:
                info.append("")
        #상장일자
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[2]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[5]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")

        #거래량
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[7]/dd/span/span[1]').text)
        except:
            info.append("")
        #시가총액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[8]/dd').text + "VND")
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Vietnam")
        info.append("Vietnam, Asia")
        info.append("")
        info.append("")

        #이벤트
        info.append("")

        #화폐
        info.append("VND")
        info.append("Vietnamese Dong")

        #관리
        info.append("Chris")
        info.append("Investing.com")
        info.append("2022-07-04")

        print(info)
        writing.writerow(info)
    except:
        print("fail")
        failList2.append(sub)
            
print(failList2)

C:\Users\user\AppData\Local\Temp\ipykernel_7392\2904818936.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


pass
pass
pass
pass
['VNM', 'Vietnam', 'Vietnam', 'UTC+07:00', '아시아', '1047000000000 USD', '96208984', '동남아시아', 'VN000000CTG7', 'Vietnam JSCmmercial Bank for Industry and Trade (CTG)', 'Vietnam JSCmmercial Bank for Industry and Trade (CTG)', "Vietnam Joint Stock Commercial Bank for Industry and Trade provides various banking products and services to individuals and organizations in Vietnam and internationally. It offers savings and current accounts; and custody, overdraft, auto-investment term, multi-purpose investment term, and flexible investment term deposits. The company also provides consumption, car, mortgage, study abroad, and business loans; short, mid, and long-term loans; special loans; international credit program loans; and overseas remittance, foreign currency trading, and wire transfer services. In addition, it offers foreign exchange services and derivatives; payment and cash management services; life insurance products; and internet banking services, as well as investme

pass
pass
pass
['VNM', 'Vietnam', 'Vietnam', 'UTC+07:00', '아시아', '1047000000000 USD', '96208984', '동남아시아', 'VN000000VGT7', 'Vietnam National Textile and Garment Group (VGT)', 'Vietnam National Textile and Garment Group (VGT)', "Vietnam National Textile and Garment Group (Vinatex) is a Vietnam-based company primarily engaged in the textile industry. Via the management of various subsidiaries, the Company is involved in the operation of fabric and lace mills, the production of men and women's apparel, as well as the wholesale of diversified textile products. Besides, the Company operates as a cut and sew apparel contractor for both local and foreign customers. Other regular business activities of the Company and its subsidiaries include textile research conducting, sewing school managing, as well as warehouses and storage facilities leasing.", 'Vietnam National Textile and Garment Group (VGT)(English: Vietnam National Textile and Garment Group (VGT)) is a public company that is listed on

pass
pass
pass
['VNM', 'Vietnam', 'Vietnam', 'UTC+07:00', '아시아', '1047000000000 USD', '96208984', '동남아시아', 'VN000000AAA4', 'An Phat Plastic and Green Environment JSC (AAA)', 'An Phat Plastic and Green Environment JSC (AAA)', "An Phat Bioplastics Joint Stock Company, formerly known as An Phat Plastic and Green Environment Joint Stock Company is a Vietnam-based company engaged in the manufacture of plastic products. The Company’s products include t-shirt plastic bags, plastic films, food-covered bags, nylon films, bags on roll, garbage bags and die cut handle bags. It also provides printing and advertising on packages and offers other products, such as plastic cutting and packing machinery, plastic printer and flow machinery. The Company has three factories in Vietnam. During the year ended December 31, 2012, 92.5% of the Company's revenue came from the sale of finished products. As of December 31, 2012, the Company had one subsidiary named An Phat - Yen Bai Mineral & Plastic Joint Stock

pass
pass
pass
['VNM', 'Vietnam', 'Vietnam', 'UTC+07:00', '아시아', '1047000000000 USD', '96208984', '동남아시아', 'VN000000DLG2', 'Duc Long Gia Lai Group JSC (DLG)', 'Duc Long Gia Lai Group JSC (DLG)', 'Duc Long Gia Lai Group Joint Stock Company is a Vietnam-based manufacturer of wood furniture. The Company’s products include indoor and outdoor furniture, such as tables, chairs, desks, beds, television cabinets, floor-boards and wardrobes. These products are produced for domestic customers as well as export. The Company is also engaged in granite and basalt stone mining and processing for export and domestic demand as well as operation of bus stations, passenger transportation, hotel and resort services. In addition, the Company is expanding its business to rubber planting and processing, mineral mining and processing including metallic and non-metallic minerals, road infrastructure construction, hydroelectricity generation investment and real estate construction and development. On March 20,

...

pass
pass
pass
['VNM', 'Vietnam', 'Vietnam', 'UTC+07:00', '아시아', '1047000000000 USD', '96208984', '동남아시아', 'VN000000PGT9', 'P TAXI SAIGON (PGT)', 'P TAXI SAIGON (PGT)', 'PGT Holdings JSC is a Vietnam-based company primarily engaged in microfinancing activities. Through its Myanmar-based subsidiary, BMF Microfinance Co Ltd, the Company is engaged in the provision of financial services to low-income clients. The Company is also involved in labor supply activities and hotel operations. In addition, it provides consulting services for mergers and acquisitions and investment activities. Other businesses include the provision of vocational training and tourism services, as well as real estate activities.', 'P TAXI SAIGON (PGT)(English: P TAXI SAIGON (PGT)) is a public company that is listed on Vietnam Stock Exchange.', 'P TAXI SAIGON (PGT)(English: P TAXI SAIGON (PGT)) is a public company that is listed on Vietnam Stock Exchange.', None, None, '', '', '', '84 28 668 40446', '-', '', 'pgt-hol

pass
pass
pass
['VNM', 'Vietnam', 'Vietnam', 'UTC+07:00', '아시아', '1047000000000 USD', '96208984', '동남아시아', 'VN000000HPM2', 'HOANG PHUC Mineral Trading and Construction JSC (HPM)', 'HOANG PHUC Mineral Trading and Construction JSC (HPM)', "HOANG PHUC Mineral Trading and Construction Joint Stock Company is a Vietnam-based company, which is mainly engaged in the wholesale of construction materials, such as crushed stones and limestone. Besides, the Company partakes in the mining and trading of stones, iron ores and other minerals. The Company's major markets include Lang Son and other northern provinces.", 'HOANG PHUC Mineral Trading and Construction JSC (HPM)(English: HOANG PHUC Mineral Trading and Construction JSC (HPM)) is a public company that is listed on Vietnam Stock Exchange.', 'HOANG PHUC Mineral Trading and Construction JSC (HPM)(English: HOANG PHUC Mineral Trading and Construction JSC (HPM)) is a public company that is listed on Vietnam Stock Exchange.', None, None, '', '', '', 

pass
pass
pass
['VNM', 'Vietnam', 'Vietnam', 'UTC+07:00', '아시아', '1047000000000 USD', '96208984', '동남아시아', 'VN000000KDM2', 'New Residential Urban Development Holdings Corp (KDM)', 'New Residential Urban Development Holdings Corp (KDM)', 'New Residential Urban Development Holdings Corp, formerly Le Gia Investment Group JSC, is a Vietnam-based company operating in the construction materials industry. The Company is engaged in the manufacture and trading of construction materials, including unfired clay bricks and concrete pipes. It is also engaged in the development of industrial parks, residential areas and urban areas. Besides, the Company is involved in ground freight transportation and the construction of residential and institutional buildings. Other activities include the installation of electrical systems, plumbing and drainage systems and heating systems, as well as metals and metal ores wholesale trading.', 'New Residential Urban Development Holdings Corp (KDM)(English: New Resi

pass
pass
pass
['VNM', 'Vietnam', 'Vietnam', 'UTC+07:00', '아시아', '1047000000000 USD', '96208984', '동남아시아', 'VN000000PIC1', 'PC3 Investment JSC (PIC)', 'PC3 Investment JSC (PIC)', 'PC3 - Investment Joint Stock Company (PC3-INVEST) is a Vietnam-based company operating in the energy industry. The Company is engaged in the management of the 14-megawatt (MW) Dak Pone hydropower plant located in Kon Plong district, Kon Tum province. The electricity output of the power plant is directly transmitted into the national grid system. Besides, the Company partakes in the development of other hydropower plants in the central region of Vietnam.', 'PC3 Investment JSC (PIC)(English: PC3 Investment JSC (PIC)) is a public company that is listed on Vietnam Stock Exchange.', 'PC3 Investment JSC (PIC)(English: PC3 Investment JSC (PIC)) is a public company that is listed on Vietnam Stock Exchange.', None, None, '', '', '', '84 2 362 212545', '84 2 362 221000', '', 'www.pc3invest.vn', '', '', '', '', '', '78A

pass
pass
pass
['VNM', 'Vietnam', 'Vietnam', 'UTC+07:00', '아시아', '1047000000000 USD', '96208984', '동남아시아', 'VN000000STS8', 'Saigon Transport Agency JSC (STS)', 'Saigon Transport Agency JSC (STS)', 'Sai Gon Transport Agency Joint-Stock Company (Tranaco) is a Vietnam-based company, with major undertakings as the operation of marine port and the provision of multimodal transportation services. The Company partakes in the management of Tan Thuan Dong port located in district 7, Ho Chi Minh city, and its regular business activities include marine cargo handling, vessel towing, ship navigating and warehouse operating. Besides, Tranaco is engaged in the ground and marine transportation of heavy goods, as well as the leasing of commercial real estate properties.', 'Saigon Transport Agency JSC (STS)(English: Saigon Transport Agency JSC (STS)) is a public company that is listed on Vietnam Stock Exchange.', 'Saigon Transport Agency JSC (STS)(English: Saigon Transport Agency JSC (STS)) is a public

In [12]:
print(len(failList2))

413


In [13]:
cover = open('Vietnam_2022_investing_2022-07-04_Chris - 3.csv','w', newline='', encoding='CP949')
path = r'C:\Users\user\Desktop\자료\chromedriver'
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드','현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])

791

In [14]:
failList3 = []
info = []
driver = webdriver.Chrome(path)
base_url = "https://www.investing.com"
driver.get('https://www.investing.com/equities/vietnam')
time.sleep(2)
try:
    driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
except:
    print("pass")
time.sleep(1)
    
for sub in failList2:
    try:
        info = []
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        time.sleep(1)
        driver.get(sub)
        info = []
        time.sleep(4)
        #이름
        name = driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text
        #국가코드
        info.append("VNM") 
        info.append("Vietnam") 
        info.append("Vietnam") 
        info.append("UTC+07:00") 
        info.append("아시아") 
        info.append("1047000000000 USD") 
        info.append("96208984") 
        info.append("동남아시아") 

        #기업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text) 
        except:
            info.append("VNMHBR" + name) 
        info.append(name)
        info.append(name)
        time.sleep(1)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]/a').click()
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        try:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Vietnam Stock Exchange."
            description_long = info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/p').text) 
        except:
            description_short = name + "(English: " + name + ")" + " is a public company that is listed on Vietnam Stock Exchange."
            description_long =  name + "(English: " + name + ")" + " is a public company that is listed on Vietnam Stock Exchange. They are providing products/services in Vietnam, and they are operated as public company."

        #기업 + 기업연락처 + 기업주소 + 기업추가정보 + 담당자정보
        try: 
            address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Vietnam"
            address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[2]').text + ", Vietnam, Asia"
            post = address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[3]').text
        except: 
            try:
                address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Vietnam"
                address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[1]/span[3]/a/span[1]').text + ", Vietnam, Asia"
                post = "N/A"
            except:
                address1 = "Vietnam"
                address2 = "Vietnam, Asia"
                post = "N/A"
        info.append(description_short)
        info.append(description_short)
        info.append(description_long)
        info.append(description_long)
        info.append("")
        info.append("")
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[4]/span[3]/a').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append(address1)
        info.append(address1)
        info.append(address2)
        info.append(address2)
        info.append(post) 

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[3]/p').text) 
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("Listed") 
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[1]').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/section/section[1]/table/tbody/tr[1]/td[1]/span').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/table/tbody/tr[1]/td[4]').text)
        except:
            info.append("Director")
        info.append("Executive Board")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[3]/span[3]').text) 
        except:
            info.append("")
        info.append("")
        try :
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[10]/div[1]/div[2]/span[3]').text) 
        except:
            info.append("")
        time.sleep(4)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #재무정보
        try:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[3]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        except:
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[4]/a').click()
            time.sleep(4)
            driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[3]/a').click()
        time.sleep(3)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(3)
        try:
            driver.find_element(By.XPATH, value='/html/body/div[5]/div[2]/i').click()
        except:
            print("pass")
        #회계연도
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[2]/span').text) 
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[1]/tr/th[3]/span').text) 
            except:
                info.append("")
        #유동자산금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. VND ")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text  + "M. VND ")
            except:
                info.append("")
        #비유동자산금액
        info.append("")
        #자산총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. VND ")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text  + "M. VND")
            except:
                info.append("")
        #유동부채금액
        try: 
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[5]/td[3]').text  + "M. VND")
            except:
                info.append("")
        #비유동부채금액
        info.append("")
        #부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[3]').text + "M. VND")
            except:
                info.append("")
        #자본총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[9]/td[3]').text + "M. VND")
            except:
                info.append("")
        #자본부채총계금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text  + "M. VND")
            except:
                info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[2]/a').click()
        time.sleep(2)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(2)
        #매출액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[1]/td[3]').text + "M. VND")
            except:
                info.append("")
        #매출원가금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[3]/td[3]').text + "M. VND")
            except:
                info.append("")
        #판매비관리비금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td/div/table/tbody/tr[1]/td[3]').text + "M. VND")
            except:
                info.append("")
        #영업이익손실금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[7]/td[2]').text + "M. VND")
            except:
                info.append("")
        #금융수익금액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[2]').text + "M. VND")
        except:
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[8]/td[3]').text + "M. VND")
            except:
                info.append("")
        #금융비용금액
        info.append("")
        #기타영업외수익금액
        info.append("")
        #기타영업외비용금액
        info.append("")
        #법인세차감전순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. VND")
            except:
                info.append("")
        #법인세비용
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[12]/td[3]').text + "M. VND")
            except:
                info.append("")
        #당기순이익
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[13]/td[3]').text + "M. VND")
            except:
                info.append("")

        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[4]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a[1]').click()
        time.sleep(5)
        #현금흐름
        #영업
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[2]/td[3]').text + "M. VND")
            except:
                info.append("")
        #투자
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[4]/td[3]').text + "M. VND")
            except:
                info.append("")
        #재무
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[6]/td[3]').text + "M. VND")
            except:
                info.append("")
        #기초기말
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[10]/td[3]').text + "M. VND")
            except:
                info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[2]').text + "M. VND")
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[9]/table/tbody[2]/tr[11]/td[3]').text + "M. VND")
            except:
                info.append("")
        #재무정보중 계산예정
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        #산업군
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[1]/li[1]/a').click()
        time.sleep(5)
        driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[5]/nav/ul/li[2]').click()
        time.sleep(5)

        try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            except:
                info.append("Public Company")
                info.append("Public Company")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
            info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[1]/a').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/section/div[8]/div[2]/a').text)
            except:
                info.append("Product and Services")
                info.append("Product and Services")


        #언어
        info.append("VI")
        info.append("Vietnamese")
        info.append("Tieng Viet")

        #주식시장정보
        driver.find_element(By.XPATH, value='/html/body/div[4]/section/ul[2]/li[1]/a').click()
        time.sleep(2)
        info.append("VSE")
        info.append("Vietnam Stock Exchange")
        info.append("Vietnam Stock Exchange")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[1]/h1').text)
        except:
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[1]/div[4]/ul/li[3]/span[2]').text)
            except:
                info.append("")
        #상장일자
        info.append("")

        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[2]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[5]/dd').text)
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")

        #거래량
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[7]/dd/span/span[1]').text)
        except:
            info.append("")
        #시가총액
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div/div[2]/div/div/div[2]/main/div/div[7]/div/div[2]/dl/div[8]/dd').text + "VND")
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Vietnam")
        info.append("Vietnam, Asia")
        info.append("")
        info.append("")

        #이벤트
        info.append("")

        #화폐
        info.append("VND")
        info.append("Vietnamese Dong")

        #관리
        info.append("Chris")
        info.append("Investing.com")
        info.append("2022-07-04")

        print(info)
        writing.writerow(info)
    except:
        print("fail")
        failList3.append(sub)
            
print(failList3)

C:\Users\user\AppData\Local\Temp\ipykernel_7392\1715466340.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


pass
pass
pass
fail
pass
pass
pass
pass
['VNM', 'Vietnam', 'Vietnam', 'UTC+07:00', '아시아', '1047000000000 USD', '96208984', '동남아시아', 'VN000000NVL0', 'No Va Land Investment Group Corp (NVL)', 'No Va Land Investment Group Corp (NVL)', 'No Va Land Investment Group Corporation, together with its subsidiaries, engages in real estate business in Vietnam. The company engages in the resident, hospitality, and industrial real estate development; real estate trading; civil and industrial construction; and provision of design and management consultancy services, as well as real estate brokerage services. It is also involved in the leasing of factories and warehouses; management consulting, short-term accommodation, botanical and zoological garden and nature reserve, and land use right auction activities, as well as real estate consulting and auction activities; and building picnic resorts. The company was formerly known as Thanh Nhon Trading Limited. No Va Land Investment Group Corporation was fou

pass
pass
pass
['VNM', 'Vietnam', 'Vietnam', 'UTC+07:00', '아시아', '1047000000000 USD', '96208984', '동남아시아', 'VN000000GAS3', 'Petrovietnam Gas JSC (GAS)', 'Petrovietnam Gas JSC (GAS)', 'PetroVietnam Gas Joint Stock Corporation engages in gathering, transporting, processing, storing, distributing, and trading gas and related products in Vietnam and internationally. It offers liquefied petroleum gas, dry gas, condensate, compressed natural gas, liquefied natural gas, and steel pipes. The company also provides gas gathering, importation, and transportation and distribution services, as well as maintenance and repair services for gas facility equipment; bottling LPG into tanks and trucks; coating and steel pipe services; and trading port, warehouse, yards, and other services. It supplies its gas to power and fertilizer plants, industrial manufacturers, LPG trading and distributing companies, and transportation and processing industries, as well as to manufacturers of ceramic tiles, steel, an

pass
pass
pass
['VNM', 'Vietnam', 'Vietnam', 'UTC+07:00', '아시아', '1047000000000 USD', '96208984', '동남아시아', 'VN000000MSN4', 'Masan Group Corp (MSN)', 'Masan Group Corp (MSN)', 'Masan Group Corporation, an investment holding company, manufactures and distributes food and beverage products in Vietnam. The company operates through five segments: Branded Consumer Products; MEATLife; High-Tech Materials; Consumer Retail; and Others. It provides soya sauce, fish sauce, seasoning, chili sauce, instant noodles, instant congee, instant coffee, instant cereals, bottled beverages, processed meat, and beer. The company offers its food and beverage products under the CHIN-SU, Nam Ngu, Tam Thai Tu, Omachi, Kokomi, Lovmi, Komi, Heo Cao Boi, Ponnie, Vinacafé, Wake-Up, Compact, Vinh Hao, Quang Hanh, Vivant, Faith, and Su Tu Trang brands. It also operates WinCommerce, a retail platform. In addition, it is involved in the trading of animal protein; manufacturing and trading of homecare products; agricultu

pass
pass
pass
['VNM', 'Vietnam', 'Vietnam', 'UTC+07:00', '아시아', '1047000000000 USD', '96208984', '동남아시아', 'VN000000TCO1', 'Duyen Hai Multi Modal Transport JSC (TCO)', 'Duyen Hai Multi Modal Transport JSC (TCO)', 'Duyen Hai Multi Modal Transport Joint Stock Company operates in the field of multimodal transportation business in Vietnam. The company offers inland clearance port; logistic services, including railway transport, air and ocean freight, and road transport services; distribution centers, and CFS and inland warehouses; P.T.I, electrify refrigerated, container repair, load/upload container, container clearning, and container lifting services; customs clearance services; and other services. Duyen Hai Multi Modal Transport Joint Stock Company was founded in 1995 and is based in Hai Phong, Vietnam.', 'Duyen Hai Multi Modal Transport JSC (TCO)(English: Duyen Hai Multi Modal Transport JSC (TCO)) is a public company that is listed on Vietnam Stock Exchange.', 'Duyen Hai Multi Modal Tr

...

pass
pass
pass
['VNM', 'Vietnam', 'Vietnam', 'UTC+07:00', '아시아', '1047000000000 USD', '96208984', '동남아시아', 'VN000000THS1', 'Song Da Thanh Hoa JSC (THS)', 'Song Da Thanh Hoa JSC (THS)', 'Song Da - Thanh Hoa Joint Stock Company engages in the wholesale of household goods. It offers furniture for school; tables and chairs; leather seats; safes; stainless steel furniture; and sofas. The company was founded in 2004 and is headquartered in Thanh Hoá, Vietnam.', 'Song Da Thanh Hoa JSC (THS)(English: Song Da Thanh Hoa JSC (THS)) is a public company that is listed on Vietnam Stock Exchange.', 'Song Da Thanh Hoa JSC (THS)(English: Song Da Thanh Hoa JSC (THS)) is a public company that is listed on Vietnam Stock Exchange.', None, None, '', '', '', '84 3 738 52175', '84 3 738 55750', '', 'thanhhoasongda.com.vn', '', '', '', '', '', '25 Le Loi Avenue Lam Son Ward, Vietnam', '25 Le Loi Avenue Lam Son Ward, Vietnam', '', '', '', '352', '', '', 'Listed', 'Le Van Tuong', '', 'General manager and Member 

pass
pass
pass
['VNM', 'Vietnam', 'Vietnam', 'UTC+07:00', '아시아', '1047000000000 USD', '96208984', '동남아시아', 'VN000000PNC1', 'Phuong Nam Cultural JSC (PNC)', 'Phuong Nam Cultural JSC (PNC)', 'Phuong Nam Cultural Joint Stock Corporation is a Vietnam-based specialty retailer. Through its retail outlets, the Company is engaged in the trading of books, magazines, stationeries, toys, video, printing materials, photographic supplies and other consumer products. It is also involved in the publication and printing of books, periodicals and catalogs, as well as the manufacture of office stationeries, labels and notebooks. In addition, the Company produces and markets movies, music records, television advertisements and other media products for entertainment purposes. The Company organizes entertainment events, operates cinemas and offers copyright protection and artistic design services.', 'Phuong Nam Cultural JSC (PNC)(English: Phuong Nam Cultural JSC (PNC)) is a public company that is listed on

pass
pass
pass
['VNM', 'Vietnam', 'Vietnam', 'UTC+07:00', '아시아', '1047000000000 USD', '96208984', '동남아시아', 'VN000000SRC6', 'Sao Vang Rubber JSC (SRC)', 'Sao Vang Rubber JSC (SRC)', 'Sao Vang Rubber Joint Stock Company manufactures and markets tires and tubes in Vietnam. It offers tires and tubes for aircraft, automobiles, bicycles, electric bicycles, and motorcycles, as well as technical rubber products. The company was founded in 1960 and is headquartered in Hanoi, Vietnam.', 'Sao Vang Rubber JSC (SRC)(English: Sao Vang Rubber JSC (SRC)) is a public company that is listed on Vietnam Stock Exchange.', 'Sao Vang Rubber JSC (SRC)(English: Sao Vang Rubber JSC (SRC)) is a public company that is listed on Vietnam Stock Exchange.', None, None, '', '', '', '84 243 858 3656', '84 243 858 3644', '', 'www.src.com.vn', '', '', '', '', '', '231 Nguyen Trai Phuong Dinh Ward, Vietnam', '231 Nguyen Trai Phuong Dinh Ward, Vietnam', '', '', '', '844', '', '', 'Listed', 'Thi Yen Pham', '', 'Member of Co

pass
pass
pass
['VNM', 'Vietnam', 'Vietnam', 'UTC+07:00', '아시아', '1047000000000 USD', '96208984', '동남아시아', 'VN000000TIX9', 'Tan Binh Import Export JSC (TIX)', 'Tan Binh Import Export JSC (TIX)', 'Tan Binh Import Export Joint Stock Company engages in the real estate business in Vietnam. The company is involved in manufacturing, trading, import-export, and investment services. It also provides construction investment services; housing business; industrial park infrastructure; and goods purchase and sale services. Tan Binh Import Export Joint Stock Company is based in Ho Chi Minh City, Vietnam.', 'Tan Binh Import Export JSC (TIX)(English: Tan Binh Import Export JSC (TIX)) is a public company that is listed on Vietnam Stock Exchange.', 'Tan Binh Import Export JSC (TIX)(English: Tan Binh Import Export JSC (TIX)) is a public company that is listed on Vietnam Stock Exchange.', None, None, '', '', '', '84 12 6458 7452', '-', '', 'www.tanimex.com.vn', '', '', '', '', '', '989/21 Hoang Dieu Stre

pass
pass
pass
['VNM', 'Vietnam', 'Vietnam', 'UTC+07:00', '아시아', '1047000000000 USD', '96208984', '동남아시아', 'VN000000VMS7', 'Vietnam Maritime Development JSC (VMS)', 'Vietnam Maritime Development JSC (VMS)', 'Vietnam Maritime Development Joint Stock Company (Vimadeco) is a Vietnam-based company, which is mainly engaged in the operation of inland container depots (ICDs) and warehouses near various marine harbors in Hai Phong city and Ho Chi Minh city. Other regular business activities of the Company include marine cargo handling, container leasing and repairing, as well as general freight transporting. Besides, the Company partakes in the provision of customs brokerage services for marine transportation, the commerce of container materials, as well as the wholesale of petroleum products.', 'Vietnam Maritime Development JSC (VMS)(English: Vietnam Maritime Development JSC (VMS)) is a public company that is listed on Vietnam Stock Exchange.', 'Vietnam Maritime Development JSC (VMS)(English:

In [15]:
print(len(failList3))

231
